In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
import pickle
from googletrans import Translator

In [11]:
import pandas as pd
df = pd.read_csv("remedy_ticket_2.csv")

In [6]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [12]:
df = df.dropna()
df['Notes'] = df['Notes'].str.replace('\n', '')
df

,Incident ID,Notes,Assigned Group
0,INC000006669671,"Problem Description: Hallo, ich kann bei der ...",GL-FSC-ESH-INT-Syneris
1,INC000006668209,Problem Description: Unable to login to Syner...,GL-FSC-ESH-Syneris
2,INC000006665046,"Problem Description: Hello, pleas set he ID 1...",GL-FSC-ESH-INT-Syneris
3,INC000006663461,Problem Description: Kindly give admin rights...,GL-FSC-ESH-INT-Syneris
4,INC000006658889,Problem Description: Es kann kein Bericht unt...,GL-FSC-ESH-Syneris
...,...,...,...
916,INC000003909549,Summary: Accident ManagementProblem Descripti...,GL-SCM-ESH-Syneris
917,INC000003895849,Summary: Accident ManagementProblem Descripti...,GL-SCM-ESH-Syneris
918,INC000003784198,( ) Incident Management System (ESH-IMs)( ) Au...,GL-SCM-ESH-Syneris
919,INC000003783938,( ) Incident Management System (ESH-IMs)( ) Au...,GL-SCM-ESH-Syneris


In [13]:
filter_data = df[df['Notes'].str.startswith('Problem Description:')].index
df = df[df.index.isin(filter_data)].reset_index(drop = True)
df['Notes'] = [i.split('Affected report:')[0].strip("Problem Description: ") for i in df['Notes']]
df

,Incident ID,Notes,Assigned Group
0,INC000006669671,"Hallo, ich kann bei der Inspection nicht unte...",GL-FSC-ESH-INT-Syneris
1,INC000006668209,Unable to login to Syneris TEST env,GL-FSC-ESH-Syneris
2,INC000006665046,"Hello, pleas set he ID 1431616 back to in work...",GL-FSC-ESH-INT-Syneris
3,INC000006663461,Kindly give admin rights on the audit manageme...,GL-FSC-ESH-INT-Syneris
4,INC000006658889,Es kann kein Bericht unter Audit Management BC...,GL-FSC-ESH-Syneris
...,...,...,...
389,INC000004034427,ase delete ITEM RCA876105,GL-SCM-ESH-Syneris
390,INC000004029718,atensatz löschen: Finding 853033 (wurde doppel...,GL-SCM-ESH-Syneris
391,INC000003955000,ase delete this item: FINDING853029,GL-SCM-ESH-Syneris
392,INC000003953290,User is no longer receiving any Syneris Emails...,GL-SCM-ESH-Syneris


In [14]:
from googletrans import Translator
translator = Translator()
df['Notes'] = [translator.translate(i).text for i in df['Notes']]
df

,Incident ID,Notes,Assigned Group
0,INC000006669671,"Hello, I cannot select the ""Ladders or Ladders...",GL-FSC-ESH-INT-Syneris
1,INC000006668209,Unable to login to Syneris TEST env,GL-FSC-ESH-Syneris
2,INC000006665046,"Hello, pleas set the ID 1431616 back to in wor...",GL-FSC-ESH-INT-Syneris
3,INC000006663461,Kindly give admin rights on the audit manageme...,GL-FSC-ESH-INT-Syneris
4,INC000006658889,No report can be dragged under Audit Managemen...,GL-FSC-ESH-Syneris
...,...,...,...
389,INC000004034427,ase delete ITEM RCA876105,GL-SCM-ESH-Syneris
390,INC000004029718,Delete data record: Finding 853033 (was entere...,GL-SCM-ESH-Syneris
391,INC000003955000,ase delete this item: FINDING853029,GL-SCM-ESH-Syneris
392,INC000003953290,User is no longer receiving any Syneris Emails...,GL-SCM-ESH-Syneris


In [15]:
lst = []
for i in df['Assigned Group']:
    if 'INT' in str(i).split('-'):
        x = True
    else:
        x = False
    lst.append(x)
df['INT'] = lst
df['EXT'] = [not i for i in lst]
df = df.drop(columns=['Assigned Group'])
df = df.astype({'INT':'int'}).astype({'EXT':'int'})
df

,Incident ID,Notes,INT,EXT
0,INC000006669671,"Hello, I cannot select the ""Ladders or Ladders...",1,0
1,INC000006668209,Unable to login to Syneris TEST env,0,1
2,INC000006665046,"Hello, pleas set the ID 1431616 back to in wor...",1,0
3,INC000006663461,Kindly give admin rights on the audit manageme...,1,0
4,INC000006658889,No report can be dragged under Audit Managemen...,0,1
...,...,...,...,...
389,INC000004034427,ase delete ITEM RCA876105,0,1
390,INC000004029718,Delete data record: Finding 853033 (was entere...,0,1
391,INC000003955000,ase delete this item: FINDING853029,0,1
392,INC000003953290,User is no longer receiving any Syneris Emails...,0,1


In [9]:
# df.to_csv('remedy_data.csv', index=False)
# df = pd.read_csv('remedy_data.csv')

In [19]:
data = df

temp = []
max = 0 
for i in data["Notes"]:
  leng = len(list(i))
  if leng > max:
    max = leng
  temp.append(list(i))

x = temp
y = data.drop(labels = ['Incident ID', 'Notes'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=1000)
with open("fit.txt", "wb") as fp:
  pickle.dump(x_train, fp)

tokenizer = Tokenizer(10000)
tokenizer.fit_on_texts(x_train)
Xcnn_train = tokenizer.texts_to_sequences(x_train)
Xcnn_test = tokenizer.texts_to_sequences(x_test)
vocab_size = len(tokenizer.word_index) + 1

maxlen = max
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)

embedding_dim = 200
textcnnmodel = Sequential()
textcnnmodel.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
textcnnmodel.add(layers.Conv1D(128, 5, activation='relu'))
textcnnmodel.add(layers.GlobalMaxPooling1D())
textcnnmodel.add(layers.Dense(10, activation='relu'))
textcnnmodel.add(layers.Dense(2, activation='sigmoid'))
textcnnmodel.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])
textcnnmodel.summary()

textcnnmodel.fit(Xcnn_train, y_train,
                     epochs=25,
                     verbose=2,
                     validation_data=(Xcnn_test, y_test),
                     batch_size=10)
loss, accuracy = textcnnmodel.evaluate(Xcnn_train, y_train, verbose=2)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = textcnnmodel.evaluate(Xcnn_test, y_test, verbose=2)
print("Testing Accuracy:  {:.4f}".format(accuracy))

# textcnnmodel.save('cnn_model.h5')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 985, 200)          15600     
                                                                 
 conv1d_2 (Conv1D)           (None, 981, 128)          128128    
                                                                 
 global_max_pooling1d_2 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 10)                1290      
                                                                 
 dense_5 (Dense)             (None, 2)                 22        
                                                                 
Total params: 145,040
Trainable params: 145,040
Non-trainable params: 0
________________________________________________

In [18]:
textcnnmodel.save('cnn_model.h5')

In [12]:
max

985

In [39]:
desc = "1293098 - Finding deadline can`t be postponed  When trying to set e.g. root cause to completed system shows that  \"Deadline may not be later than the deadline of the finding (1403291): 1/31/2023\" When trying to postpone the deadline of the finding system shows \"Deadline may not be later than the deadline of the finding (1403291): 1/31/2023\" although this is the deadline of the finding."
with open("fit.txt", "rb") as fp:
    b = pickle.load(fp)
tokenizer = Tokenizer(100000)
tokenizer.fit_on_texts(b)
Xcnn_train = tokenizer.texts_to_sequences([desc])
maxlen = max
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)

res = textcnnmodel.predict(Xcnn_train, verbose=0)
if str(res.argmax(axis=1)) == '[0]':
    print("Prediction: Internal\tConfidence:", res[0][0])
else:
    print("Prediction: External\tConfidence:", res[0][1])

Prediction: External	Confidence: 0.7971987


In [34]:
res[0][0]

0.120910875